In [169]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 100)
import warnings
warnings.filterwarnings('ignore')
import gc

In [170]:
grid_df = pd.read_pickle('../data/input/grid_2015.pkl')
grid_df = grid_df.sample(frac=0.01)
grid_df.head()
#grid_df = grid_df[(grid_df.tm_m >=4)&(grid_df.tm_m<=6)]

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end
34709048,HOBBIES_2_069_TX_3_validation,HOBBIES_2_069,HOBBIES_2,HOBBIES,TX_3,TX,1541,1.0,0,2.970703,3.169922,2.970703,0.050232,2.984375,0.937012,2.0,119,1.0,0.987305,1.000000,NaN,NaN,NaN,NaN,0,0,0,18,16,4,4,3,5,1
41122602,FOODS_2_070_CA_1_validation,FOODS_2_070,FOODS_2,FOODS,CA_1,CA,1753,0.0,132,5.980469,6.359375,5.339844,0.400635,6.007812,0.940430,4.0,71,1.0,0.988281,0.979492,NaN,NaN,NaN,NaN,0,0,0,16,47,11,4,3,0,0
40780840,HOUSEHOLD_2_192_WI_2_validation,HOUSEHOLD_2_192,HOUSEHOLD_2,HOUSEHOLD,WI_2,WI,1741,0.0,110,1.969727,1.969727,1.919922,0.018738,1.961914,1.000000,2.0,115,1.0,1.005859,1.000000,NaN,NaN,NaN,NaN,1,0,0,4,45,11,4,1,2,0
37367432,FOODS_1_197_TX_2_validation,FOODS_1_197,FOODS_1,FOODS,TX_2,TX,1629,0.0,0,5.281250,5.281250,3.970703,0.341064,4.871094,1.000000,4.0,13,1.0,1.093750,1.000000,NaN,NaN,NaN,NaN,0,1,1,15,29,7,4,3,2,0
40632946,FOODS_3_381_WI_3_validation,FOODS_3_381,FOODS_3,FOODS,WI_3,WI,1736,0.0,145,2.980469,2.980469,2.880859,0.037933,2.962891,1.000000,2.0,206,1.0,1.000000,1.021484,NaN,NaN,NaN,NaN,0,0,0,30,44,10,4,5,4,0


In [171]:
grid_df.shape

(110299, 34)

In [172]:
#grid_df.groupby(['store_id', 'item_id'])['sell_price'].apply(lambda x: round(x, 1).mode())

In [173]:
cat_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 
            'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI' ]
num_cols = ['release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 
            'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y']
data_col = ['d']
target_col =['sales']
unuse_cols = ['id', 'item_id', 'cat_id', 'store_id', 'event_name_2', 'event_type_2']

In [174]:
import category_encoders as ce
list_cols = list(set(cat_cols) - set(unuse_cols))
ce_ohe = ce.OneHotEncoder(cols=list_cols,handle_unknown='impute')

In [175]:
%%time
use_cols = [col for col in grid_df.columns if col not in unuse_cols]
grid_df_one_hot = ce_ohe.fit_transform(grid_df)
grid_df_one_hot.head()

CPU times: user 748 ms, sys: 400 ms, total: 1.15 s
Wall time: 1.22 s


,id,item_id,dept_id_1,dept_id_2,dept_id_3,dept_id_4,dept_id_5,dept_id_6,dept_id_7,cat_id,store_id,state_id_1,state_id_2,state_id_3,d,sales,release,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y,event_name_1_1,event_name_1_2,event_name_1_3,event_name_1_4,event_name_1_5,event_name_1_6,event_name_1_7,event_name_1_8,event_name_1_9,event_name_1_10,event_name_1_11,event_name_1_12,event_name_1_13,event_name_1_14,event_name_1_15,event_name_1_16,event_name_1_17,event_name_1_18,event_name_1_19,event_name_1_20,event_name_1_21,event_name_1_22,event_name_1_23,event_name_1_24,event_name_1_25,event_name_1_26,event_name_1_27,event_name_1_28,event_name_1_29,event_name_1_30,event_name_1_31,event_type_1_1,event_type_1_2,event_type_1_3,event_type_1_4,event_type_1_5,event_name_2,event_type_2,snap_CA_1,snap_CA_2,snap_TX_1,snap_TX_2,snap_WI_1,snap_WI_2,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end
34709048,HOBBIES_2_069_TX_3_validation,HOBBIES_2_069,1,0,0,0,0,0,0,HOBBIES,TX_3,1,0,0,1541,1.0,0,2.970703,3.169922,2.970703,0.050232,2.984375,0.937012,2.0,119,1.0,0.987305,1.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,NaN,NaN,1,0,1,0,1,0,18,16,4,4,3,5,1
41122602,FOODS_2_070_CA_1_validation,FOODS_2_070,0,1,0,0,0,0,0,FOODS,CA_1,0,1,0,1753,0.0,132,5.980469,6.359375,5.339844,0.400635,6.007812,0.940430,4.0,71,1.0,0.988281,0.979492,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,NaN,NaN,1,0,1,0,1,0,16,47,11,4,3,0,0
40780840,HOUSEHOLD_2_192_WI_2_validation,HOUSEHOLD_2_192,0,0,1,0,0,0,0,HOUSEHOLD,WI_2,0,0,1,1741,0.0,110,1.969727,1.969727,1.919922,0.018738,1.961914,1.000000,2.0,115,1.0,1.005859,1.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,NaN,NaN,0,1,1,0,1,0,4,45,11,4,1,2,0
37367432,FOODS_1_197_TX_2_validation,FOODS_1_197,0,0,0,1,0,0,0,FOODS,TX_2,1,0,0,1629,0.0,0,5.281250,5.281250,3.970703,0.341064,4.871094,1.000000,4.0,13,1.0,1.093750,1.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,NaN,NaN,1,0,0,1,0,1,15,29,7,4,3,2,0
40632946,FOODS_3_381_WI_3_validation,FOODS_3_381,0,0,0,0,1,0,0,FOODS,WI_3,0,0,1,1736,0.0,145,2.980469,2.980469,2.880859,0.037933,2.962891,1.000000,2.0,206,1.0,1.000000,1.021484,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,NaN,NaN,1,0,1,0,1,0,30,44,10,4,5,4,0


In [176]:
use_oh_cols = [col for col in grid_df_one_hot.columns if col not in unuse_cols]
grid_for_ridge = grid_df_one_hot[use_oh_cols]

In [177]:
#grid_for_ridge[grid_for_ridge.isnull()]

In [178]:
grid_for_ridge.fillna(0, inplace=True)

In [197]:
TARGET      = 'sales'            
START_TRAIN = 0                  
END_TRAIN   = 1750               
P_HORIZON   = 28        

In [198]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

grid_train = grid_for_ridge[grid_for_ridge.d <= END_TRAIN]
grid_valid = grid_for_ridge[grid_for_ridge.d > END_TRAIN]

X_train = grid_train.drop(TARGET, axis=1)
y_train = grid_train[TARGET]
X_valid = grid_valid.drop(TARGET, axis=1)
y_valid = grid_valid[TARGET]

# X = grid_for_ridge.drop(TARGET, axis=1)
# y = grid_for_ridge[TARGET]

sc = StandardScaler()
X[num_cols] = sc.fit_transform(X[num_cols])

In [199]:
# X_train = X[X.d <= END_TRAIN]
# y_train = y[:X_train.shape[0]]
# X_valid = X[X.d > END_TRAIN]
# y_train = y[-len(X_valid):]

In [200]:
y_valid.shape

(14674,)

In [201]:
#X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, shuffle=False)

ridge = Ridge()
ridge.fit(X_train, y_train)

y_preds = ridge.predict(X_valid)
score = mean_squared_error(y_valid, y_preds)
print(score)

13.206502917496856


In [202]:
print(ridge.intercept_) 
print(ridge.coef_) 

2.9133578922751817
[-9.60281625e-01  1.69071680e-01 -5.97807727e-01  9.42456433e-02
  1.04172408e+00 -1.45946912e-01  3.98994860e-01 -6.89283954e-02
  1.48879298e-01 -7.99509023e-02 -2.71002321e-03 -1.46854442e-03
 -4.84003324e-01  1.98973674e-02  6.47890101e-02 -5.29281415e-01
  2.92872303e-01 -2.11780755e+00  2.79146778e-02 -3.81220756e-03
  5.02162388e-02  3.53009999e+00  1.07719867e+00  3.25452274e-02
 -1.57612658e-01  1.23286071e-02  1.11477774e-01 -1.53510801e-01
 -5.01443075e-03 -5.62494757e-02  0.00000000e+00  1.14638292e-01
  1.94510999e-02  1.90612161e-01  3.51807977e-01 -1.28033671e-01
 -1.70627496e-01 -2.07095690e-01 -4.36664596e-01  2.89640575e-01
 -2.93800884e-01  1.90441549e-01  1.83508090e-01  2.47819227e-01
 -9.82438992e-02 -1.60004436e-01  1.31790481e-01 -1.17610009e-01
 -1.68693929e-01  0.00000000e+00  4.68928846e-02  1.51839158e-01
  7.83688731e-02  0.00000000e+00  3.25452274e-02 -1.44378783e-01
 -4.57689076e-02  2.53572301e-01 -9.59698373e-02 -7.59743112e-03
  7.59

In [203]:
print(len(ridge.coef_), len(grid_for_ridge.columns))

72 73


In [204]:
coef_df = pd.DataFrame({'columns': grid_for_ridge.drop('sales', axis=1).columns, 'importance': ridge.coef_})
coef_df.head(20)

,columns,importance
0,dept_id_1,-0.960282
1,dept_id_2,0.169072
2,dept_id_3,-0.597808
3,dept_id_4,0.094246
4,dept_id_5,1.041724
5,dept_id_6,-0.145947
6,dept_id_7,0.398995
7,state_id_1,-0.068928
8,state_id_2,0.148879
9,state_id_3,-0.079951
